In [1]:
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
findspark.init()
spark=SparkSession.builder.appName("SparkSQL").getOrCreate()    

your 131072x1 screen size is bogus. expect trouble
24/12/16 13:49:31 WARN Utils: Your hostname, SAADSLAP resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/16 13:49:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/16 13:49:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


**LOAN DATASET**

In [7]:
loan_df=spark.read.csv("files/loan.csv",inferSchema=True,header=True)
loan_df.show()

+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|Customer_ID|Age|Gender|         Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|     Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|    IB14001| 30|  MALE|       BANK MANAGER|        SINGLE|          4| 50000|      22199|            6|           HOUSING| 10,00,000 |      5|      42,898|               6|                 9|
|    IB14008| 44|  MALE|          PROFESSOR|       MARRIED|          6| 51000|      19999|            4|          SHOPPING|     50,000|      3|      33,999|               1|                 5|
|    IB14012| 30|FEMALE|           

In [8]:
loan_df.printSchema()

root
 |-- Customer_ID: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- Marital Status: string (nullable = true)
 |-- Family Size: integer (nullable = true)
 |-- Income: integer (nullable = true)
 |-- Expenditure: integer (nullable = true)
 |-- Use Frequency: integer (nullable = true)
 |-- Loan Category: string (nullable = true)
 |-- Loan Amount: string (nullable = true)
 |-- Overdue: integer (nullable = true)
 |--  Debt Record: string (nullable = true)
 |--  Returned Cheque: integer (nullable = true)
 |--  Dishonour of Bill: integer (nullable = true)



In [9]:
len(loan_df.columns)

15

In [10]:
loan_df.count()

500

In [11]:
loan_df.distinct().count()

500

In [13]:
loan_df.describe().show()

24/12/14 22:06:57 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----------+-----------------+------+---------------+--------------+----------------+-----------------+------------------+------------------+-------------+-----------+------------------+-----------------+-----------------+------------------+
|summary|Customer_ID|              Age|Gender|     Occupation|Marital Status|     Family Size|           Income|       Expenditure|     Use Frequency|Loan Category|Loan Amount|           Overdue|      Debt Record|  Returned Cheque| Dishonour of Bill|
+-------+-----------+-----------------+------+---------------+--------------+----------------+-----------------+------------------+------------------+-------------+-----------+------------------+-----------------+-----------------+------------------+
|  count|        500|              500|   500|            500|           500|             500|              468|               481|               500|          500|        500|               500|              500|              500|               5

In [ ]:
#Number of loans in each category
loan_df.groupBy('Loan Category').count().orderBy("count",ascending=False).show()

+------------------+-----+
|     Loan Category|count|
+------------------+-----+
|         GOLD LOAN|   77|
|           HOUSING|   67|
|        AUTOMOBILE|   60|
|        TRAVELLING|   53|
|       RESTAURANTS|   41|
|COMPUTER SOFTWARES|   35|
|          SHOPPING|   35|
|          BUSINESS|   24|
|  EDUCATIONAL LOAN|   20|
|        RESTAURANT|   20|
|           DINNING|   14|
|       ELECTRONICS|   14|
|   HOME APPLIANCES|   14|
|       AGRICULTURE|   12|
|       BOOK STORES|    7|
|          BUILDING|    7|
+------------------+-----+



In [18]:
#Numbe of people who have taken loan mora than 1 lack loan
loan_df.filter(loan_df['Loan Amount']>"1,00,000").count() 

379

In [19]:
#People with income greater than 60000 dollars
loan_df.filter(loan_df['Income']>60000).count()

198

In [22]:
#number of people with income less than 50000 and have 2 or more returned cheque
loan_df.filter((loan_df[' Returned Cheque']>1) & (loan_df['Income']<50000)).count()

137

In [28]:
#number of people with 2 or more returned cheque and are single
loan_df.filter((loan_df[' Returned Cheque']>1) & (loan_df['Marital Status']=='SINGLE')).count()

111

In [31]:
#number of people with expenditure over 50000 and income more than 50000  by each category
loan_df.filter((loan_df["Expenditure"]>50000)& (loan_df['Income']>50000)).groupBy('Loan Category').count().show()

+-------------+-----+
|Loan Category|count|
+-------------+-----+
|      HOUSING|    3|
|   TRAVELLING|    1|
|   AUTOMOBILE|    1|
|      DINNING|    1|
+-------------+-----+



**CREDIT CARD DATASET**

In [32]:
credit_df=spark.read.csv("files/credit card.csv",inferSchema=True,header=True)

In [33]:
credit_df.printSchema()

root
 |-- RowNumber: integer (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- Surname: string (nullable = true)
 |-- CreditScore: integer (nullable = true)
 |-- Geography: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- Balance: double (nullable = true)
 |-- NumOfProducts: integer (nullable = true)
 |-- IsActiveMember: integer (nullable = true)
 |-- EstimatedSalary: double (nullable = true)
 |-- Exited: integer (nullable = true)



In [34]:
credit_df.show(5)

+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|RowNumber|CustomerId| Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|        1|  15634602|Hargrave|        619|   France|Female| 42|     2|      0.0|            1|             1|      101348.88|     1|
|        2|  15647311|    Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|             1|      112542.58|     0|
|        3|  15619304|    Onio|        502|   France|Female| 42|     8| 159660.8|            3|             0|      113931.57|     1|
|        4|  15701354|    Boni|        699|   France|Female| 39|     1|      0.0|            2|             0|       93826.63|     0|
|        5|  15737888|Mitchell|        850|    Spain|Female| 4

In [36]:
len(credit_df.columns)

13

In [38]:
credit_df.count()

10000

In [39]:
#number of members eligible for a credit card
credit_df.filter(credit_df['CreditScore']>700).count()

3116

In [43]:
#number of members who are eligible for a credit card and are active by geography
credit_df.filter((credit_df['CreditScore']>700) & (credit_df['IsActiveMember']==1)).groupBy('Geography').count().show()

+---------+-----+
|Geography|count|
+---------+-----+
|  Germany|  397|
|   France|  834|
|    Spain|  406|
+---------+-----+



In [46]:
#credit card users in Spain and are (active or credit score >700)
credit_df.filter((credit_df['Geography']=='Spain')& ((credit_df['IsActiveMember']==1) | (credit_df['CreditScore']>700))).count()


1668

In [47]:
credit_df.filter(credit_df['Geography']=="Spain").count()

2477

In [50]:
credit_df.filter((credit_df["EstimatedSalary"]>100000) & (credit_df['Exited']==1)).count()

1044

**Transaction Dataset**

In [3]:
txn_df=spark.read.csv("files/txn.csv",inferSchema=True,header=True)

In [4]:
txn_df.printSchema()

root
 |-- Account No: string (nullable = true)
 |-- TRANSACTION DETAILS: string (nullable = true)
 |-- VALUE DATE: string (nullable = true)
 |--  WITHDRAWAL AMT : double (nullable = true)
 |--  DEPOSIT AMT : double (nullable = true)
 |-- BALANCE AMT: double (nullable = true)



In [5]:
#Count of transaction on every account
txn_df.groupBy('Account No').count().orderBy("count",ascending=False).show()

+-------------+-----+
|   Account No|count|
+-------------+-----+
|     1196428'|48779|
|409000362497'|29840|
|409000438620'|13454|
|     1196711'|10536|
|409000493210'| 6014|
|409000438611'| 4588|
|409000611074'| 1093|
|409000493201'| 1044|
|409000425051'|  802|
|409000405747'|   51|
+-------------+-----+



In [9]:
#Maximum withdrawal Amount
txn_df.groupBy("Account No").max(' Withdrawal AMT ').orderBy("max( Withdrawal AMT )", ascending=False).show()

+-------------+---------------------+
|   Account No|max( Withdrawal AMT )|
+-------------+---------------------+
|     1196711'|        4.594475464E8|
|409000438620'|                4.0E8|
|409000425051'|               3.54E8|
|409000438611'|                2.4E8|
|409000405747'|                1.7E8|
|     1196428'|                1.5E8|
|409000362497'|        1.413662392E8|
|409000493210'|                1.5E7|
|409000493201'|            2500000.0|
|409000611074'|             912000.0|
+-------------+---------------------+



In [11]:
#Minimum withdrawal amount of an account
txn_df.groupBy('Account No').min(' Withdrawal AMT ').orderBy("min( Withdrawal AMT )", ascending=False).show()

+-------------+---------------------+
|   Account No|min( Withdrawal AMT )|
+-------------+---------------------+
|409000611074'|                120.0|
|409000405747'|                 21.0|
|409000493201'|                  2.1|
|409000425051'|                 1.25|
|409000362497'|                 0.97|
|409000438620'|                 0.34|
|     1196711'|                 0.25|
|     1196428'|                 0.25|
|409000438611'|                  0.2|
|409000493210'|                 0.01|
+-------------+---------------------+



In [13]:
#Maximum deposit of an account
txn_df.groupBy('Account No').max(' Deposit AMT ').orderBy("max( Deposit AMT )", ascending=False).show()

+-------------+------------------+
|   Account No|max( Deposit AMT )|
+-------------+------------------+
|409000438620'|           5.448E8|
|     1196711'|             5.0E8|
|     1196428'|     2.119594422E8|
|409000405747'|           2.021E8|
|409000362497'|             2.0E8|
|409000438611'|          1.7025E8|
|409000493210'|             1.5E7|
|409000425051'|             1.5E7|
|409000611074'|         3000000.0|
|409000493201'|         1000000.0|
+-------------+------------------+



In [15]:
#Minimum deposit ammount of an account
txn_df.groupBy('Account No').min(' Deposit AMT ').orderBy("min( Deposit AMT )",ascending=False).show()

+-------------+------------------+
|   Account No|min( Deposit AMT )|
+-------------+------------------+
|409000611074'|            1320.0|
|409000405747'|             500.0|
|     1196711'|              1.01|
|     1196428'|               1.0|
|409000425051'|               1.0|
|409000493201'|               0.9|
|409000438620'|              0.07|
|409000438611'|              0.03|
|409000362497'|              0.03|
|409000493210'|              0.01|
+-------------+------------------+



In [17]:
#sum of balance in every bank account
txn_df.groupBy('Account No').sum('Balance AMT').show()

+-------------+--------------------+
|   Account No|    sum(Balance AMT)|
+-------------+--------------------+
|409000438611'|-2.49486577068339...|
|     1196711'|-1.60476498101275E13|
|     1196428'| -8.1418498130721E13|
|409000493210'|-3.27584952132095...|
|409000611074'|       1.615533622E9|
|409000425051'|-3.77211841164998...|
|409000405747'|-2.43108047067000...|
|409000493201'|1.0420831829499985E9|
|409000438620'|-7.12291867951358...|
|409000362497'| -5.2860004792808E13|
+-------------+--------------------+



In [18]:
#Number of transaction on each date
txn_df.groupBy('Value Date').count().orderBy('count',ascending=False).show()

+----------+-----+
|Value Date|count|
+----------+-----+
| 27-Jul-17|  567|
| 13-Aug-18|  463|
|  8-Nov-17|  402|
|  7-Oct-17|  382|
| 10-Jul-18|  374|
| 12-Dec-17|  367|
| 12-Sep-18|  365|
|  9-Aug-18|  360|
| 19-Sep-17|  358|
| 16-Mar-17|  353|
| 10-Sep-18|  344|
| 14-Jul-17|  333|
|  7-Mar-18|  319|
| 11-Oct-18|  303|
| 22-Aug-17|  301|
|  9-Jan-18|  299|
|  9-Oct-18|  297|
| 20-Apr-18|  296|
|  9-Jul-18|  292|
|  7-Apr-18|  291|
+----------+-----+
only showing top 20 rows



In [19]:
#list of customers with withdrawal amount more than 100000
txn_df.select("Account No", "TRANSACTION DETAILS"," WITHDRAWAL AMT ").filter(txn_df[' WITHDRAWAL AMT ']>100000).show()

+-------------+--------------------+----------------+
|   Account No| TRANSACTION DETAILS| WITHDRAWAL AMT |
+-------------+--------------------+----------------+
|409000611074'|INDO GIBL Indiafo...|        133900.0|
|409000611074'|INDO GIBL Indiafo...|        195800.0|
|409000611074'|INDO GIBL Indiafo...|        143800.0|
|409000611074'|INDO GIBL Indiafo...|        331650.0|
|409000611074'|INDO GIBL Indiafo...|        129000.0|
|409000611074'|INDO GIBL Indiafo...|        230013.0|
|409000611074'|INDO GIBL Indiafo...|        367900.0|
|409000611074'|INDO GIBL Indiafo...|        108000.0|
|409000611074'|INDO GIBL Indiafo...|        141000.0|
|409000611074'|INDO GIBL Indiafo...|        206000.0|
|409000611074'|INDO GIBL Indiafo...|        242300.0|
|409000611074'|INDO GIBL Indiafo...|        113250.0|
|409000611074'|INDO GIBL Indiafo...|        206900.0|
|409000611074'|INDO GIBL Indiafo...|        276000.0|
|409000611074'|INDO GIBL Indiafo...|        171000.0|
|409000611074'|INDO GIBL Ind